<a href="https://colab.research.google.com/github/AliaksandrSiarohin/articulated-animation/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Demo for paper "Motion Representations for Articulated Animation"
**Clone repository. First install git lfs. Pulling all checkpoints may take a while**


In [1]:
## commented out, only for colab
# !curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
# !sudo apt-get install git-lfs
# !git lfs install
# !git clone https://github.com/snap-research/articulated-animation/
# cd articulated-animation

## xvdp modifcations
    redirected weights - as general rule I store weights in separate mount/ volume. etc.
    added if scikit-learn.__version__ >= 0.19
    added yaml.SafeLoader
    tested image from the wild

### Verdicts. A lot of memorization exists in this form of learning. 
    Changing background color, driven person size, hair color, introduces errors.

In [4]:
! conda env list | grep "*"

abj                   *  /home/z/mambaforge/envs/abj


In [5]:
import os
import os.path as osp
dataroot = 'checkpoints' # if weigths set as original
dataroot = osp.expanduser("~/weights/articulated-animation")
print(dataroot, osp.isdir(dataroot))
print('current dir', os.getcwd())

/home/z/weights/articulated-animation True
current dir /home/z/work/gits/Animation/articulated-animation


**Load source image and driving video.**

In [6]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('sup-mat/source.png')
driving_video = imageio.mimread('sup-mat/driving.mp4')


source_image = resize(source_image, (384, 384))[..., :3]
driving_video = [resize(frame, (384, 384))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


**Load checkpoints.**

In [7]:
print(source_image.shape)

(384, 384, 3)


In [8]:
from demo import load_checkpoints

generator, region_predictor, avd_network = load_checkpoints(config_path='config/ted384.yaml',
                                                            checkpoint_path=osp.join(dataroot, 'ted384.pth'))

**Perform animation.**

In [9]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, 
                             region_predictor, avd_network, animation_mode='avd')

#save resulting video
imageio.mimsave('generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|█████████████████████| 133/133 [00:05<00:00, 23.38it/s]


## test on image from the wild

In [11]:
url = 'https://static.jutarnji.hr/images/live-multimedia/binary/2019/8/22/10/kate%20moss.jpg'
source_image2 = imageio.imread(url)

In [12]:
source_image2.shape

(1024, 798, 3)

In [23]:
sz = max(source_image2.shape)
pad_im = np.zeros((sz,sz, 3), dtype=source_image2.dtype)
h,w = source_image2.shape[:2]
h2 = (sz - h)//2
w2 = (sz - w)//2
pad_im[h2:h2+h, w2:w2+w, :] = source_image2

pad_im = resize(pad_im, (384, 384))[..., :3]

In [25]:
pad_im.shape

(384, 384, 3)

In [26]:
predictions = make_animation(pad_im, driving_video, generator, 
                             region_predictor, avd_network, animation_mode='avd')

#save resulting video
imageio.mimsave('generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(pad_im, driving_video, predictions).to_html5_video())

100%|█████████████████████| 133/133 [00:05<00:00, 23.39it/s]
